In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time

In [ ]:
from util_results import Results

In [3]:
from te_datasim.jointprocess import MVJointProcessSimulator
from te_datasim.lineargaussian import MVLinearGaussianSimulator

In [4]:
import torch; torch.set_printoptions(sci_mode=None)
# Check if CUDA is available
if torch.cuda.is_available():
    compute_device = torch.device("cuda")
    print("CUDA is available. Using GPU.")
else:
    compute_device = torch.device("cpu")
    print("CUDA is not available. Using CPU.")

CUDA is available. Using GPU.


In [5]:
from agm_te.model import _train_agm
from agm_te.dataset import DataSet
from agm_te.estimate import init_agms_from_loaders

In [6]:
def get_dataset(generator, n_samples, batch_size, seed):
    data_dict = {'X':[], 'Y':[]}
    
    X, Y = generator.simulate(n_samples, seed=seed)
    for i in range(0, n_samples, batch_size):
        data_dict['X'].append(X[i:i+batch_size])
        data_dict['Y'].append(Y[i:i+batch_size])
    data = DataSet(data_dict)
    return data

In [7]:
EPOCHS = 1000
LR = 0.01
L2P = 0.00
OPT = 'sgd'

In [8]:
def TE_agmte(dataset, device, var_from, var_to, hidden_size, batch_size, plot_loss=False):
    smoothing = 100
    
    tl_dataloader_1, tl_dataloader_2 = dataset.get_TE_dataloaders(var_from=var_from, var_to=var_to)
    tl_model_1, tl_model_2 = init_agms_from_loaders(tl_dataloader_1, tl_dataloader_2, dyn_model_type='MLPTanh', hidden_size=hidden_size, num_layers=2)
    tl_model_1.to(device)
    tl_model_2.to(device)
    tl_model_1, loss_1 = _train_agm(tl_model_1, tl_dataloader_1,
                                    batch_size=batch_size, epochs=EPOCHS, learning_rate=0.01, optimize='sgd')
    print()
    tl_model_2, loss_2 = _train_agm(tl_model_2, tl_dataloader_2, 
                                    batch_size=batch_size, epochs=EPOCHS, learning_rate=0.01, optimize='sgd')
    print()
    if plot_loss:
        plt.figure(figsize=(12, 4))
        plt.plot(loss_1, alpha=0.5, color='blue')
        plt.plot(loss_2, alpha=0.5, color='orange')

    smooth_loss_1 = np.array([np.mean(loss_1[i:i+smoothing]) for i in range(0, len(loss_1)-smoothing)])
    smooth_loss_2 = np.array([np.mean(loss_2[i:i+smoothing]) for i in range(0, len(loss_2)-smoothing)])
    
    if plot_loss:
        plt.plot(range(smoothing, len(loss_1)), smooth_loss_1, color='blue', linewidth=2, label='Loss 1')
        plt.plot(range(smoothing, len(loss_1)), smooth_loss_2, color='orange', linewidth=2, label='Loss 2')
        plt.legend()
        plt.show()
    
    TE = np.round(smooth_loss_1[-1]-smooth_loss_2[-1],4)
    print(f"TE: {var_from}->{var_to} {TE}")
    return TE

# Basic Validity

In [9]:
REPLICATES = 5
SAMPLE_SIZE = 10000
BATCH_SIZE = 500
NB = 1

## Linear Gaussian

In [10]:
# Specify the range of lambda values to test
lg_lambda_range = list(np.linspace(0, 1, 9, endpoint=True))

# Initialize the list of generators with one for each lambda value
lg_generator_lst = [MVLinearGaussianSimulator(n_dim=1, coupling=lam) for lam in lg_lambda_range]

# get the reference values
lg_TE_X2Y_ref_lst = [generator.analytic_transfer_entropy('X', 'Y') for generator in lg_generator_lst]
lg_TE_Y2X_ref_lst = [generator.analytic_transfer_entropy('Y', 'X') for generator in lg_generator_lst]

In [11]:
lg_results_TE_X2Y = Results(columns=['method', 'coupling'])
lg_results_TE_Y2X = Results(columns=['method', 'coupling'])

for r in range(REPLICATES):
    print(f"\n### REPLICATE {r+1}/{REPLICATES} ###\n")
    for lam, generator in zip(lg_lambda_range, lg_generator_lst):
        print("# Coupling = ", lam, "True TE X->Y = ", generator.analytic_transfer_entropy('X', 'Y'), "True TE Y->X = ", generator.analytic_transfer_entropy('Y', 'X'))
        # Simulate data
        dataset = get_dataset(generator, SAMPLE_SIZE, BATCH_SIZE, seed=r)
        # Estimate X -> Y
        TE_X2Y = TE_agmte(dataset, compute_device, 'X', 'Y', 16, NB)
        lg_results_TE_X2Y.write(method='agmte', coupling=lam, value=TE_X2Y)
        # Estimate Y -> X
        TE_Y2X = TE_agmte(dataset, compute_device, 'Y', 'X', 16, NB)
        lg_results_TE_Y2X.write(method='agmte', coupling=lam, value=TE_Y2X)

lg_results_TE_X2Y.df.to_csv('results/agmte/lg_results_TE_X2Y_bv.csv', index=False)
lg_results_TE_Y2X.df.to_csv('results/agmte/lg_results_TE_Y2X_bv.csv', index=False)


### REPLICATE 1/5 ###

# Coupling =  0.0 True TE X->Y =  0.0 True TE Y->X =  0.0
Epoch [990/1000], Loss: 0.59990708
Epoch [990/1000], Loss: 0.59973125
TE: X->Y 0.0002
Epoch [990/1000], Loss: 0.60939604
Epoch [990/1000], Loss: 0.60893184
TE: Y->X 0.0005
# Coupling =  0.125 True TE X->Y =  0.0 True TE Y->X =  0.0092
Epoch [990/1000], Loss: 0.60000257
Epoch [990/1000], Loss: 0.59968642
TE: X->Y 0.0003
Epoch [990/1000], Loss: 0.61861269
Epoch [990/1000], Loss: 0.60882527
TE: Y->X 0.0098
# Coupling =  0.25 True TE X->Y =  0.0 True TE Y->X =  0.0356
Epoch [990/1000], Loss: 0.60000213
Epoch [990/1000], Loss: 0.59975647
TE: X->Y 0.0002
Epoch [990/1000], Loss: 0.64449712
Epoch [990/1000], Loss: 0.60909259
TE: Y->X 0.0354
# Coupling =  0.375 True TE X->Y =  0.0 True TE Y->X =  0.0763
Epoch [990/1000], Loss: 0.60000004
Epoch [990/1000], Loss: 0.59968157
TE: X->Y 0.0003
Epoch [990/1000], Loss: 0.68509194
Epoch [990/1000], Loss: 0.60878105
TE: Y->X 0.0763
# Coupling =  0.5 True TE X->Y =  0.0 True

In [12]:
# Specify the range of lambda values to test
jp_lambda_range = list(np.linspace(-3, 3, 9, endpoint=True))

# Initialize the list of generators with one for each lambda value
jp_generator_lst = [MVJointProcessSimulator(n_dim=1, lam=lam) for lam in jp_lambda_range]

# get the reference values
jp_TE_X2Y_ref_lst = [generator.analytic_transfer_entropy('X', 'Y') for generator in jp_generator_lst]
jp_TE_Y2X_ref_lst = [generator.analytic_transfer_entropy('Y', 'X') for generator in jp_generator_lst]

In [13]:
jp_results_TE_X2Y = Results(columns=['method', 'coupling'])
jp_results_TE_Y2X = Results(columns=['method', 'coupling'])

for r in range(REPLICATES):
    print(f"\n### REPLICATE {r+1}/{REPLICATES} ###\n")
    for lam, generator in zip(jp_lambda_range, jp_generator_lst):
        print("# Coupling = ", lam, "True TE X->Y = ", generator.analytic_transfer_entropy('X', 'Y'), "True TE Y->X = ", generator.analytic_transfer_entropy('Y', 'X'))
        # Simulate data
        dataset = get_dataset(generator, SAMPLE_SIZE, BATCH_SIZE, seed=r)
        # Estimate X -> Y
        TE_X2Y = TE_agmte(dataset, compute_device, 'X', 'Y', 16, NB)
        jp_results_TE_X2Y.write(method='agmte', coupling=lam, value=TE_X2Y)
        # Estimate Y -> X
        TE_Y2X = TE_agmte(dataset, compute_device, 'Y', 'X', 16, NB)
        jp_results_TE_Y2X.write(method='agmte', coupling=lam, value=TE_Y2X)

jp_results_TE_X2Y.df.to_csv('results/agmte/jp_results_TE_X2Y_bv.csv', index=False)
jp_results_TE_Y2X.df.to_csv('results/agmte/jp_results_TE_Y2X_bv.csv', index=False)


### REPLICATE 1/5 ###

# Coupling =  -3.0 True TE X->Y =  0.8292 True TE Y->X =  0.0
Epoch [990/1000], Loss: 1.40761787
Epoch [990/1000], Loss: 0.57996737
TE: X->Y 0.8276
Epoch [990/1000], Loss: 1.41203045
Epoch [990/1000], Loss: 1.41183854
TE: Y->X 0.0002
# Coupling =  -2.25 True TE X->Y =  0.8202 True TE Y->X =  0.0
Epoch [990/1000], Loss: 1.40814165
Epoch [990/1000], Loss: 0.59516223
TE: X->Y 0.8127
Epoch [990/1000], Loss: 1.41204264
Epoch [990/1000], Loss: 1.41192353
TE: Y->X 0.0001
# Coupling =  -1.5 True TE X->Y =  0.7749 True TE Y->X =  0.0
Epoch [990/1000], Loss: 1.40796404
Epoch [990/1000], Loss: 0.64315374
TE: X->Y 0.7644
Epoch [990/1000], Loss: 1.41204413
Epoch [990/1000], Loss: 1.41181412
TE: Y->X 0.0002
# Coupling =  -0.75 True TE X->Y =  0.6422 True TE Y->X =  0.0
Epoch [990/1000], Loss: 1.40410725
Epoch [990/1000], Loss: 0.77391904
TE: X->Y 0.6298
Epoch [990/1000], Loss: 1.41201511
Epoch [990/1000], Loss: 1.41185322
TE: Y->X 0.0002
# Coupling =  0.0 True TE X->Y =  0.41

# Sample size scaling

In [15]:
lg_generator = MVLinearGaussianSimulator(n_dim=1, coupling=0.5)
jp_generator = MVJointProcessSimulator(n_dim=1, lam=0.0)
sample_sizes = [500, 1000, 5000, 10000, 50000, 100000]

In [16]:
lg_results_TE_X2Y = Results(columns=['method', 'sample_size'])
lg_results_TE_Y2X = Results(columns=['method', 'sample_size'])

for r in range(REPLICATES):
    print(f"\n### REPLICATE {r+1}/{REPLICATES} ###\n")
    for samples in sample_sizes:
        print("# Samples = ", samples, "#")
        # Simulate data
        dataset = get_dataset(lg_generator, samples, int(np.round(samples/20)), seed=r)
        # Estimate X -> Y
        TE_X2Y = TE_agmte(dataset, compute_device, 'X', 'Y', 16, NB)
        lg_results_TE_X2Y.write(method='agmte', sample_size=samples, value=TE_X2Y)
        # Estimate Y -> X
        TE_Y2X = TE_agmte(dataset, compute_device, 'Y', 'X', 16, NB)
        lg_results_TE_Y2X.write(method='agmte', sample_size=samples, value=TE_Y2X)

lg_results_TE_X2Y.df.to_csv('results/agmte/lg_results_TE_X2Y_ss.csv', index=False)
lg_results_TE_Y2X.df.to_csv('results/agmte/lg_results_TE_Y2X_ss.csv', index=False)


### REPLICATE 1/5 ###

# Samples =  500 #


Epoch [990/1000], Loss: 0.57833002
Epoch [990/1000], Loss: 0.56550968
TE: X->Y 0.0126
Epoch [990/1000], Loss: 0.68325674
Epoch [990/1000], Loss: 0.56250367
TE: Y->X 0.1208
# Samples =  1000 #
Epoch [990/1000], Loss: 0.59310173
Epoch [990/1000], Loss: 0.58539343
TE: X->Y 0.0073
Epoch [990/1000], Loss: 0.70054152
Epoch [990/1000], Loss: 0.58913923
TE: Y->X 0.1114
# Samples =  5000 #
Epoch [990/1000], Loss: 0.59610704
Epoch [990/1000], Loss: 0.59592941
TE: X->Y 0.0002
Epoch [990/1000], Loss: 0.73392021
Epoch [990/1000], Loss: 0.60799118
TE: Y->X 0.1259
# Samples =  10000 #
Epoch [990/1000], Loss: 0.60006024
Epoch [990/1000], Loss: 0.59962094
TE: X->Y 0.0004
Epoch [990/1000], Loss: 0.73606497
Epoch [990/1000], Loss: 0.60894982
TE: Y->X 0.1271
# Samples =  50000 #
Epoch [990/1000], Loss: 0.60970547
Epoch [990/1000], Loss: 0.60960642
TE: X->Y 0.0001
Epoch [990/1000], Loss: 0.73881574
Epoch [990/1000], Loss: 0.61453434
TE: Y->X 0.1243
# Samples =  100000 #
Epoch [990/1000], Loss: 0.61212491
E

In [17]:
jp_results_TE_X2Y = Results(columns=['method', 'sample_size'])
jp_results_TE_Y2X = Results(columns=['method', 'sample_size'])

for r in range(REPLICATES):
    print(f"\n### REPLICATE {r+1}/{REPLICATES} ###\n")
    for samples in sample_sizes:
        print("# Samples = ", samples, "#")
        # Simulate data
        dataset = get_dataset(jp_generator, samples, int(np.round(samples/20)), seed=r)
        # Estimate X -> Y
        TE_X2Y = TE_agmte(dataset, compute_device, 'X', 'Y', 16, NB)
        jp_results_TE_X2Y.write(method='agmte', sample_size=samples, value=TE_X2Y)
        # Estimate Y -> X
        TE_Y2X = TE_agmte(dataset, compute_device, 'Y', 'X', 16, NB)
        jp_results_TE_Y2X.write(method='agmte', sample_size=samples, value=TE_Y2X)

jp_results_TE_X2Y.df.to_csv('results/agmte/jp_results_TE_X2Y_ss.csv', index=False)
jp_results_TE_Y2X.df.to_csv('results/agmte/jp_results_TE_Y2X_ss.csv', index=False)


### REPLICATE 1/5 ###

# Samples =  500 #
Epoch [990/1000], Loss: 1.37343058
Epoch [990/1000], Loss: 0.97759535
TE: X->Y 0.394
Epoch [990/1000], Loss: 1.38815508
Epoch [990/1000], Loss: 1.38908573
TE: Y->X -0.0011
# Samples =  1000 #
Epoch [990/1000], Loss: 1.37973456
Epoch [990/1000], Loss: 0.99105465
TE: X->Y 0.3881
Epoch [990/1000], Loss: 1.38568337
Epoch [990/1000], Loss: 1.38173084
TE: Y->X 0.0038
# Samples =  5000 #
Epoch [990/1000], Loss: 1.40149035
Epoch [990/1000], Loss: 1.00933048
TE: X->Y 0.3917
Epoch [990/1000], Loss: 1.40969162
Epoch [990/1000], Loss: 1.40762321
TE: Y->X 0.0021
# Samples =  10000 #
Epoch [990/1000], Loss: 1.40369313
Epoch [990/1000], Loss: 1.00713383
TE: X->Y 0.3962
Epoch [990/1000], Loss: 1.41201619
Epoch [990/1000], Loss: 1.41188147
TE: Y->X 0.0001
# Samples =  50000 #
Epoch [990/1000], Loss: 1.41717659
Epoch [990/1000], Loss: 1.01247535
TE: X->Y 0.4044
Epoch [990/1000], Loss: 1.41689053
Epoch [990/1000], Loss: 1.41689813
TE: Y->X -0.0
# Samples =  1000

# Dimensionality Scaling with redundant dimensions

In [19]:
dim_range = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
sample_sizes = [10000, 100000]

## Linear Gaussian

In [20]:
# Initialize the list of generators with one for each dimension
lg_generator_lst = [MVLinearGaussianSimulator(n_dim=dim, n_redundant_dim=dim-1) for dim in dim_range]
# Get the reference values
lg_TE_X2Y_ref_lst = [generator.analytic_transfer_entropy('X', 'Y') for generator in lg_generator_lst]
lg_TE_Y2X_ref_lst = [generator.analytic_transfer_entropy('Y', 'X') for generator in lg_generator_lst]

In [21]:
lg_results_TE_X2Y = Results(columns=['method', 'n_dim', 'sample_size'])
lg_results_TE_Y2X = Results(columns=['method', 'n_dim', 'sample_size'])

for r in range(REPLICATES):
    print(f"\n### REPLICATE {r+1}/{REPLICATES} ###\n")
    for dim, generator in zip(dim_range, lg_generator_lst):
        print("## Dim = ", dim, "#")
        for samples in sample_sizes:
            print("# Sample size = ", samples, "#")
            # Simulate data
            dataset = get_dataset(generator, samples, int(np.round(samples/20)), seed=r)
            # Estimate X -> Y
            TE_X2Y = TE_agmte(dataset, compute_device, 'X', 'Y', dim*16, NB)
            lg_results_TE_X2Y.write(method='agmte', n_dim=dim, sample_size=samples, value=TE_X2Y)
            # Estimate Y -> X
            TE_Y2X = TE_agmte(dataset, compute_device, 'Y', 'X', dim*16, NB)
            lg_results_TE_Y2X.write(method='agmte', n_dim=dim, sample_size=samples, value=TE_Y2X)

lg_results_TE_X2Y.df.to_csv('results/agmte/lg_results_TE_X2Y_dimred.csv', index=False)
lg_results_TE_Y2X.df.to_csv('results/agmte/lg_results_TE_Y2X_dimred.csv', index=False)


### REPLICATE 1/5 ###

## Dim =  1 #
# Sample size =  10000 #
Epoch [990/1000], Loss: 0.59994299
Epoch [990/1000], Loss: 0.59954118
TE: X->Y 0.0004
Epoch [990/1000], Loss: 0.73560051
Epoch [990/1000], Loss: 0.60894853
TE: Y->X 0.1266
# Sample size =  100000 #
Epoch [990/1000], Loss: 0.61220766
Epoch [990/1000], Loss: 0.61222325
TE: X->Y -0.0
Epoch [990/1000], Loss: 0.73679992
Epoch [990/1000], Loss: 0.61262013
TE: Y->X 0.1242
## Dim =  2 #
# Sample size =  10000 #
Epoch [990/1000], Loss: 2.02630841
Epoch [990/1000], Loss: 2.02473243
TE: X->Y 0.0015
Epoch [990/1000], Loss: 2.15166856
Epoch [990/1000], Loss: 2.02456357
TE: Y->X 0.1271
# Sample size =  100000 #
Epoch [990/1000], Loss: 2.02991453
Epoch [990/1000], Loss: 2.02986212
TE: X->Y 0.0001
Epoch [990/1000], Loss: 2.15635821
Epoch [990/1000], Loss: 2.03223515
TE: Y->X 0.1241
## Dim =  3 #
# Sample size =  10000 #
Epoch [990/1000], Loss: 3.43234438
Epoch [990/1000], Loss: 3.42852699
TE: X->Y 0.0037
Epoch [990/1000], Loss: 3.57863688


## Joint Process

In [22]:
# Initialize the list of generators with one for each dimension
jp_generator_lst = [MVJointProcessSimulator(lam = 0.0, n_dim=dim, n_redundant_dim=dim-1) for dim in dim_range]
# Get the reference values
jp_TE_X2Y_ref_lst = [generator.analytic_transfer_entropy('X', 'Y') for generator in jp_generator_lst]
jp_TE_Y2X_ref_lst = [generator.analytic_transfer_entropy('Y', 'X') for generator in jp_generator_lst]

In [23]:
jp_results_TE_X2Y = Results(columns=['method', 'n_dim', 'sample_size'])
jp_results_TE_Y2X = Results(columns=['method', 'n_dim', 'sample_size'])

for r in range(REPLICATES):
    print(f"\n### REPLICATE {r+1}/{REPLICATES} ###\n")
    for dim, generator in zip(dim_range, jp_generator_lst):
        print("## Dim = ", dim, "#")
        for samples in sample_sizes:
            print("# Sample size = ", samples, "#")
            # Simulate data
            dataset = get_dataset(generator, samples, int(np.round(samples/20)), seed=r)
            # Estimate X -> Y
            TE_X2Y = TE_agmte(dataset, compute_device, 'X', 'Y', dim*16, NB)
            jp_results_TE_X2Y.write(method='agmte', n_dim=dim, sample_size=samples, value=TE_X2Y)
            # Estimate Y -> X
            TE_Y2X = TE_agmte(dataset, compute_device, 'Y', 'X', dim*16, NB)
            jp_results_TE_Y2X.write(method='agmte', n_dim=dim, sample_size=samples, value=TE_Y2X)

jp_results_TE_X2Y.df.to_csv('results/agmte/jp_results_TE_X2Y_dimred.csv', index=False)
jp_results_TE_Y2X.df.to_csv('results/agmte/jp_results_TE_Y2X_dimred.csv', index=False)


### REPLICATE 1/5 ###

## Dim =  1 #
# Sample size =  10000 #
Epoch [990/1000], Loss: 1.40366864
Epoch [990/1000], Loss: 1.00495748
TE: X->Y 0.3984
Epoch [990/1000], Loss: 1.41204535
Epoch [990/1000], Loss: 1.41186869
TE: Y->X 0.0002
# Sample size =  100000 #
Epoch [990/1000], Loss: 1.41731446
Epoch [990/1000], Loss: 1.01335655
TE: X->Y 0.4037
Epoch [990/1000], Loss: 1.41761742
Epoch [990/1000], Loss: 1.41759189
TE: Y->X 0.0
## Dim =  2 #
# Sample size =  10000 #
Epoch [990/1000], Loss: 2.83030521
Epoch [990/1000], Loss: 2.42918275
TE: X->Y 0.4005
Epoch [990/1000], Loss: 2.82681051
Epoch [990/1000], Loss: 2.82666257
TE: Y->X 0.0001
# Sample size =  100000 #
Epoch [990/1000], Loss: 2.83480878
Epoch [990/1000], Loss: 2.42916013
TE: X->Y 0.4053
Epoch [990/1000], Loss: 2.83701331
Epoch [990/1000], Loss: 2.83696815
TE: Y->X 0.0
## Dim =  3 #
# Sample size =  10000 #
Epoch [990/1000], Loss: 4.23281361
Epoch [990/1000], Loss: 3.83241024
TE: X->Y 0.3998
Epoch [990/1000], Loss: 4.25172935
Epoc

# Dimensionality Scaling without redundant dimensions

In [25]:
dim_range = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
sample_sizes = [10000, 100000]

## Linear Gaussian

In [26]:
# Initialize the list of generators with one for each dimension
lg_generator_lst = [MVLinearGaussianSimulator(n_dim=dim) for dim in dim_range]
# Get the reference values
lg_TE_X2Y_ref_lst = [generator.analytic_transfer_entropy('X', 'Y') for generator in lg_generator_lst]
lg_TE_Y2X_ref_lst = [generator.analytic_transfer_entropy('Y', 'X') for generator in lg_generator_lst]

In [27]:
lg_results_TE_X2Y = Results(columns=['method', 'n_dim', 'sample_size'])
lg_results_TE_Y2X = Results(columns=['method', 'n_dim', 'sample_size'])

for r in range(REPLICATES):
    print(f"\n### REPLICATE {r+1}/{REPLICATES} ###\n")
    for dim, generator in zip(dim_range, lg_generator_lst):
        print("## Dim = ", dim, "#")
        for samples in sample_sizes:
            print("# Sample size = ", samples, "#")
            # Simulate data
            dataset = get_dataset(generator, samples, int(np.round(samples/20)), seed=r)
            # Estimate X -> Y
            TE_X2Y = TE_agmte(dataset, compute_device, 'X', 'Y', dim*16, NB)
            lg_results_TE_X2Y.write(method='agmte', n_dim=dim, sample_size=samples, value=TE_X2Y)
            # Estimate Y -> X
            TE_Y2X = TE_agmte(dataset, compute_device, 'Y', 'X', dim*16, NB)
            lg_results_TE_Y2X.write(method='agmte', n_dim=dim, sample_size=samples, value=TE_Y2X)

lg_results_TE_X2Y.df.to_csv('results/agmte/lg_results_TE_X2Y_dim.csv', index=False)
lg_results_TE_Y2X.df.to_csv('results/agmte/lg_results_TE_Y2X_dim.csv', index=False)


### REPLICATE 1/5 ###

## Dim =  1 #
# Sample size =  10000 #
Epoch [990/1000], Loss: 0.59989418
Epoch [990/1000], Loss: 0.59963569
TE: X->Y 0.0003
Epoch [990/1000], Loss: 0.73614373
Epoch [990/1000], Loss: 0.60895358
TE: Y->X 0.1272
# Sample size =  100000 #
Epoch [990/1000], Loss: 0.61211201
Epoch [990/1000], Loss: 0.61210904
TE: X->Y 0.0
Epoch [990/1000], Loss: 0.73671466
Epoch [990/1000], Loss: 0.61274123
TE: Y->X 0.124
## Dim =  2 #
# Sample size =  10000 #
Epoch [990/1000], Loss: 1.21084309
Epoch [990/1000], Loss: 1.20825435
TE: X->Y 0.0026
Epoch [990/1000], Loss: 1.47636128
Epoch [990/1000], Loss: 1.23029444
TE: Y->X 0.246
# Sample size =  100000 #
Epoch [990/1000], Loss: 1.22123227
Epoch [990/1000], Loss: 1.22110523
TE: X->Y 0.0001
Epoch [990/1000], Loss: 1.47910843
Epoch [990/1000], Loss: 1.22908398
TE: Y->X 0.25
## Dim =  3 #
# Sample size =  10000 #
Epoch [990/1000], Loss: 1.81824296
Epoch [990/1000], Loss: 1.81421819
TE: X->Y 0.0038
Epoch [990/1000], Loss: 2.22549062
Epoch

## Joint Process

In [28]:
# Initialize the list of generators with one for each dimension
jp_generator_lst = [MVJointProcessSimulator(n_dim=dim, lam=0.0) for dim in dim_range]
# Get the reference values
jp_TE_X2Y_ref_lst = [generator.analytic_transfer_entropy('X', 'Y') for generator in jp_generator_lst]
jp_TE_Y2X_ref_lst = [generator.analytic_transfer_entropy('Y', 'X') for generator in jp_generator_lst]

In [29]:
jp_results_TE_X2Y = Results(columns=['method', 'n_dim', 'sample_size'])
jp_results_TE_Y2X = Results(columns=['method', 'n_dim', 'sample_size'])

for r in range(REPLICATES):
    print(f"\n### REPLICATE {r+1}/{REPLICATES} ###\n")
    for dim, generator in zip(dim_range, jp_generator_lst):
        print("## Dim = ", dim, "#")
        for samples in sample_sizes:
            print("# Sample size = ", samples, "#")
            # Simulate data
            dataset = get_dataset(generator, samples, int(np.round(samples/20)), seed=r)
            # Estimate X -> Y
            TE_X2Y = TE_agmte(dataset, compute_device, 'X', 'Y', dim*16, NB)
            jp_results_TE_X2Y.write(method='agmte', n_dim=dim, sample_size=samples, value=TE_X2Y)
            # Estimate Y -> X
            TE_Y2X = TE_agmte(dataset, compute_device, 'Y', 'X', dim*16, NB)
            jp_results_TE_Y2X.write(method='agmte', n_dim=dim, sample_size=samples, value=TE_Y2X)

jp_results_TE_X2Y.df.to_csv('results/agmte/jp_results_TE_X2Y_dim.csv', index=False)
jp_results_TE_Y2X.df.to_csv('results/agmte/jp_results_TE_Y2X_dim.csv', index=False)


### REPLICATE 1/5 ###

## Dim =  1 #
# Sample size =  10000 #
Epoch [990/1000], Loss: 1.40368636
Epoch [990/1000], Loss: 1.00622459
TE: X->Y 0.3969
Epoch [990/1000], Loss: 1.41204164
Epoch [990/1000], Loss: 1.41193434
TE: Y->X 0.0001
# Sample size =  100000 #
Epoch [990/1000], Loss: 1.41729889
Epoch [990/1000], Loss: 1.01296616
TE: X->Y 0.4039
Epoch [990/1000], Loss: 1.41759848
Epoch [990/1000], Loss: 1.41758547
TE: Y->X 0.0
## Dim =  2 #
# Sample size =  10000 #
Epoch [990/1000], Loss: 2.82892699
Epoch [990/1000], Loss: 2.01031649
TE: X->Y 0.8176
Epoch [990/1000], Loss: 2.83087721
Epoch [990/1000], Loss: 2.83018205
TE: Y->X 0.0007
# Sample size =  100000 #
Epoch [990/1000], Loss: 2.83434137
Epoch [990/1000], Loss: 2.03042294
TE: X->Y 0.803
Epoch [990/1000], Loss: 2.83347221
Epoch [990/1000], Loss: 2.83340401
TE: Y->X 0.0001
## Dim =  3 #
# Sample size =  10000 #
Epoch [990/1000], Loss: 4.25337291
Epoch [990/1000], Loss: 3.00916057
TE: X->Y 1.2417
Epoch [990/1000], Loss: 4.25667743
Ep

KeyboardInterrupt: 